# ML orchestration with Seldon Core

Seldon Core allows to efficiently deploy and run machine learning models as part of containerized applications. It uses a Kubernetes-native approach to declaratively define graphs of inference servers and integrates with common monitoring and service mesh tools.

This notebook describes how to deploy the _location extractor_ service on OpenShift and set up advanced routing through Istio.

## Prerequisites

* Install the Seldon Core operator.
* Deploy a SeldonDeployment CR, see `k8s/helm-chart/templates/location-extractor-seldon.yaml`.
* Install OpenShift Service Mesh for advanced routing scenarios.

## API specification

Seldon Core exposes a generic API for inference. The location extractor service implements the following prediction interface:

### Method

`POST`

### Endpoint

`/api/v1.0/predictions`

### Request JSON body

```
{
  "data": {
    "names": [],
    "ndarray": ["input text"]
  }
}
```

### Response JSON body

```
{
  "data": {
    "names": [],
    "ndarray": [
      {
        "extracted location": "sample location string",
        "generated address": "sample address string",
        "latitude": 42.0,
        "longitude": 51.0
      }
      ...
    ]
  },
  "meta": {
    "requestPath": {
      "model-name": "image:tag"
    }
  }
}
```

Refer to [External Prediction](https://docs.seldon.io/projects/seldon-core/en/latest/reference/apis/external-prediction.html) API for further details.


## Sample request

In [ ]:
from pprint import pprint
from requests import post

Replace the `endpoint` by the location extractor's Route URL:

In [ ]:
endpoint = 'http://location-extractor-localnews-demo.apps.cluster-n7nsp.n7nsp.sandbox1852.opentlc.com'
prediction_url = f'{endpoint}/api/v1.0/predictions'

Try different `sample` strings:

In [ ]:
sample = 'My name is Clara and I live in Berkeley, California.'

payload = {'data': {'names': [], 'ndarray': [sample]}}

The following location entities were detected:

In [ ]:
response = post(prediction_url, json=payload)
pprint(response.json())

## Advanced routing

Once OpenShift Service Mesh is installed, you can integrate Seldon Core as outlined in [Working with Red Hat OpenShift Service Mesh](https://docs.seldon.io/projects/seldon-core/en/latest/ingress/openshift.html).

Istio integration offers advanced routing patterns such as shadow deployments, A/B testing, canary deployments, and Multi armed Bandits. Refer to [Ingress with Istio](